In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
dfes = pd.read_csv('RiverRoad_Quan_RF.csv', index_col=0, parse_dates=True)
dfes.head()

,qu_dem_MIN,qu_dem_MAX,qu_dem_RANGE,qu_dem_MEAN,qu_dem_STD,qu_dem_SUM,qu_dsm_MIN,qu_dsm_MAX,qu_dsm_RANGE,qu_dsm_MEAN,...,qu_curpl_SUM,qu_curpr_MIN,qu_curpr_MAX,qu_curpr_RANGE,qu_curpr_MEAN,qu_curpr_STD,qu_curpr_SUM,wetland_type,FID_RiverRd_AOI,wetland
GRID_ID,,,,,,,,,,,,,,,,,,,,,
AA-10,0.691847,1.731059,1.039212,1.117221,0.202755,313.939159,0.731374,6.256660,5.525286,2.557360,...,1688.477507,-561.943237,585.859314,1147.802551,-6.984654,173.043352,-1941.733846,nonwetland,0,nonwetland
AA-11,0.641888,1.487661,0.845773,1.059745,0.147762,295.668776,0.661373,6.679267,6.017894,2.126747,...,-1129.404510,-841.298767,679.957519,1521.256287,0.069758,168.483844,19.392675,nonwetland,0,nonwetland
AA-12,0.770032,1.507396,0.737363,1.176038,0.160304,329.290651,0.770032,5.681332,4.911300,1.700724,...,-994.710507,-614.981445,614.381775,1229.363220,-9.649778,201.707961,-2692.288038,nonwetland,0,nonwetland
AA-13,0.761148,1.413489,0.652341,1.131694,0.115760,316.874198,0.761148,1.462370,0.701221,1.130406,...,-2264.176594,-577.830017,633.895386,1211.725403,-1.244892,180.105412,-347.324849,nonwetland,0,nonwetland
AA-14,0.696000,1.392386,0.696386,1.160470,0.116413,322.610787,0.667013,1.392386,0.725372,1.158858,...,-571.203099,-574.029480,501.253113,1075.282593,-1.556956,170.969213,-435.947672,nonwetland,0,nonwetland


In [3]:
formula = 'wetland ~ qu_dem_MIN' #Put variables to the right of the tilde, and see the result at the bottom cell

In [4]:
train, test = train_test_split(dfes, test_size=.33, random_state=42)

In [5]:
model_log = smf.glm(formula = formula, data=train, family=sm.families.Binomial())
result = model_log.fit()
print(result.summary())
np.mean(dfes['qu_dem_MIN'])

                              Generalized Linear Model Regression Results                              
Dep. Variable:     ['wetland[nonwetland]', 'wetland[wetland]']   No. Observations:                 5769
Model:                                                     GLM   Df Residuals:                     5767
Model Family:                                         Binomial   Df Model:                            1
Link Function:                                           logit   Scale:                          1.0000
Method:                                                   IRLS   Log-Likelihood:                -2571.1
Date:                                         Mon, 12 Jul 2021   Deviance:                       5142.1
Time:                                                 17:14:45   Pearson chi2:                 9.65e+03
No. Iterations:                                              6                                         
Covariance Type:                                     nonrobust  

1.012418477089304

In [6]:
print("Coefficeients")
print(result.params)
print()
print("p-Values")
print(result.pvalues)
print()
print("Dependent variables")
print(result.model.endog_names)

Coefficeients
Intercept    -3.910903
qu_dem_MIN    3.570943
dtype: float64

p-Values
Intercept     1.496734e-297
qu_dem_MIN    3.622883e-198
dtype: float64

Dependent variables
['wetland[nonwetland]', 'wetland[wetland]']


In [7]:
predictions = result.predict(test)
print(predictions[1:10])

GRID_ID
AT-27    0.119907
U-10     0.872237
BD-45    0.086751
F-44     0.106011
C-83     0.118708
CL-28    0.111154
AA-90    0.300869
G-62     0.132715
AC-55    0.997717
dtype: float64


In [8]:
predictions_nominal = [ "wetland" if x < 0.5 else "nonwetland" for x in predictions]
print(predictions_nominal[1:10])

['wetland', 'nonwetland', 'wetland', 'wetland', 'wetland', 'wetland', 'wetland', 'wetland', 'nonwetland']


In [9]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(test["wetland"], 
                       predictions_nominal))

[[ 605  451]
 [  43 1743]]


In [10]:
print(classification_report(test["wetland"], 
                            predictions_nominal, 
                            digits = 3))

              precision    recall  f1-score   support

  nonwetland      0.934     0.573     0.710      1056
     wetland      0.794     0.976     0.876      1786

    accuracy                          0.826      2842
   macro avg      0.864     0.774     0.793      2842
weighted avg      0.846     0.826     0.814      2842

